<a href="https://colab.research.google.com/github/kodenshacho/SimpleVolumeRendering/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#include <thread>
#include <future>

// 예제 함수: 실제 구현에서 이미지 데이터 로딩 로직으로 대체해야 합니다.
std::vector<unsigned char> loadImageDataInBackground() {
    // 이미지 데이터 로드 로직
    std::vector<unsigned char> imageData(1024); // 예제 데이터
    return imageData;
}

void uploadTextureToOpenGL(const std::vector<unsigned char>& imageData) {
    GLuint textureID;
    glGenTextures(1, &textureID);
    glBindTexture(GL_TEXTURE_2D, textureID);

    // 여기서 imageData를 OpenGL 텍스처로 업로드
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, imageData.data());

    glBindTexture(GL_TEXTURE_2D, 0);
}

int main() {
    // 백그라운드 스레드에서 이미지 데이터 로드
    std::future<std::vector<unsigned char>> futureImageData = std::async(std::launch::async, loadImageDataInBackground);

    // 메인 스레드에서 다른 작업 수행 가능...

    // 이미지 데이터가 준비되면 OpenGL 텍스처로 업로드
    std::vector<unsigned char> imageData = futureImageData.get(); // 백그라운드 스레드로부터 이미지 데이터 받기
    uploadTextureToOpenGL(imageData); // 메인 스레드에서 OpenGL을 사용하여 텍스처 업로드

    return 0;
}